# Decodificando la estructura secundaria de COVID-19 con Python y Pandas

* * *

### 1)  ¿Qué es el ARN ?


|  ![Fosfato](Imagenes/fosfato.png) | ![Ribosa](Imagenes/ribosa.png) | ![Base nitrogenada](Imagenes/bases_nitrogenadas.png) | ![ARN](Imagenes/RAD.png) |
|:-:|:-:|:-:|:-:|
|  Grupo fosfato | Azúcar de ribosa | Base nitrogenada  | ARN VS ADN   |


La molécula de ácido ribonucleico (ARN) desempeña funciones fundamentales en biología, incluida la transmisión de información genética, la regulación de la expresión génica y la catálisis de reacciones bioquímicas. Muchas moléculas de ARN o sus partes (dominios o motivos) se pliegan en estructuras tridimensionales (3D) estables que definen, al menos parcialmente, su capacidad de interactuar con otras moléculas y llevar a cabo sus tareas dentro de la célula.


### 2)  ¿Cuáles son los niveles de organización del ARN ?


|  ![Fosfato](Imagenes/nucleotidos.png) | ![Ribosa](Imagenes/secundaria.png) | ![Base nitrogenada](Imagenes/terciaria.gif) |
|:-:|:-:|:-:|
|  Primaria | Secundaria | Terciaria  |


La molécula de ARN de una sola cadena, para poder ejercer una función necesita adoptar una estructura específica. La estructura primaria es la secuencia lineal de nucleótidos de Adenina, Guanina , Citosina y Uracilo. La estructura secundaria son todos los elementos de asa y horquilla, las bases desapareadas, bucles y estructuras centroides que se forman a partir de las interacciones por puentes de hidrógeno que se dan entre pares de bases nitrogenadas. La estructura terciaria es la conformación tridimensional de la molécula de ARN, esta emerge de las interacciones entre estructuras secundarias. Como el ARN está cargado negativamente, se necesitan iones metálicos como Mg2 + para estabilizar muchas estructuras secundarias y terciarias.

### 2) ¿Cómo modelamos la estructura secundaria del ARN?

|  AAAUAGGCUAUA | ![zucker](Imagenes/zucker.png)  | (((..((((...)))).))) | ![HAIRPIN](Imagenes/hairpin.png)|
|:-:|:-:|:-:|:-:|
| Tomamos una secuencia de ARN  | Definimos parámetros e interacciones| Construimos modelo en notacion de brackets  |Simulamos la estructura secundaria| 

* * *


### 3)  ¿Por qué modelar la estructura secundaria del ARN  de COVID19?


|  ![Fosfato](Imagenes/coronavirus.jpeg) | ![Ribosa](Imagenes/rnam.png) | ![Base nitrogenada](Imagenes/pepe.png) |
|:-:|:-:|:-:|
|  Coronavirus | ARNm | ?  |


La estructura secundaria del ARN mensajero desempeña un papel importante en la biosíntesis de proteínas. Su impacto negativo en la traducción puede reducir la tasa de síntesis de una proteína al alentar o boquear el inicio y el movimiento de los ribosomas a lo largo del ARNm, convirtiéndose en un factor importante en la regulación de la expresión génica. Varios algoritmos pueden predecir la formación de estructuras secundarias calculando la energía libre mínima de las secuencias de ARN, o hallar una secuencia de ARN para una estructura dada. Si se aumenta la formación de  estructuras secundarias de ARNm  en COVID19 se puede disminuir la tasa a la que se sintetizan sus proteínas, haciendo menos efectivo su ciclo infectivo. 


### El objetivo de este taller es:

Aprender a usar el cálculo de la MFE para predecir cambios en la estructura del ARN

### El caso de hoy:

Eres un biohacker que acaba de descubir un nuevo medicamento que tiene la habilidad de generar que las moléculas del ARN de coronavirus puedan aparearse de manera extraña. ¿Lograrás reducir la tasa de síntesis de su proteína RdRp?

![pepeee](Imagenes/pepescientist.jpeg)

## Paso 0: Instala las siguientes librerías

In [ ]:
#conda install pandas
#conda install -c bioconda viennarna
#pip install --upgrade forgi

## Paso 1: Abrimos la secuencia del genoma de COVID-19 en formato fasta

La función readgenome se encarga de "leer" la secuencia del genoma de COVID19 tal y como fue descargada de la base de datos del Instituto nacional de información para biotecnología (NCBI). En este recuadro, COVID19.fasta es la secuencia en formato fasta.

In [ ]:
def readgenome(filename):
    genome =''
    with open(filename,'r')as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readgenome('COVID19.fasta')
print("Largo en pb del genoma de COVID-19:",len(genome))
print("Secuencia de ADN:",genome)


## Paso 2: Generamos la secuencia de ARN a partir de la secuencia de ADN

In [ ]:
rna= ""
# Genera un archivo tipo string vacío en donde luego se guardara la secuencia de ARN
for i in genome:
    # Reemplaza todos los nucelótidos de Timina (T) con Uracilo (U)
    if i == "T":
        rna += "U"
    else:
        rna += i

# Imprime el string de ARN
print( "Secuencia de ARN: ", rna)

## Paso 3: Calculamos la MFE de la secuencia de ARN del dominio orf1ab de COVID-19

Calculamos la energía mínima de plegamiento del genoma de COVID19 a través de un análisis de ventana corrediza, con un tamaño de ventana de 100 nt y saltos de 1 nt. Su MFE se calcula en los primeros 21555 nt porque en esta region se encuentra el dominio de la proteína Rdrp.**

</pre>
### Para efecto práctico de este taller solo tomaremos en cuenta los primeros 1000 nucleótidos

In [ ]:
# Dominio de un orf1ab que sintetiza una poli-proteina  que contiene a Rdrp
YP_009724389_1 = rna[0:1000]
# en realidad es 21555
print('YP_009724389_1:',len(YP_009724389_1))
rna=YP_009724389_1 

In [ ]:
import RNA
fin=0
def sliding_window_analysis(genome,function, window_size=100, step_size=1):
    """Devuelve un iterador que produce tuplas (inicio, fin, propiedad).
    Donde start y end son los índices utilizados para dividir la lista de entrada
    y function es el valor de retorno de la función dada la división
    lista.
    """
    for start in range(0, len(genome), step_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])

    for start in range(0, len(genome), window_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])
        
        
def fold_energy(genome):
    'Devuelve la energía de plegamiento de la secuencia de cada ventana'
    'Devuelve la energía de plegamiento de la secuencia de cada ventana'
    import RNA
    from RNA import params_load
    params_load('rna_turner2004.par') 
    RNA.cvar.dangles = 0
    
    fc = RNA.fold_compound(genome)
    #Estas funciones calculan la estructura secundaria en notacion de brackets y la MFE
    # Particularmente, la MFE compuesta, es decir de todas las conformaciones de RNA posibles 
    # es decir te da la MFE promedio de todas las estructuras secundarias potenicales a partir de una secuencia
    (mfe_struct, mfe) = fc.mfe()
    fc.exp_params_rescale(mfe)
    #Esto genera la funcionde particion que forma parte del calculo de la MFE compuesta(promedio)
    (pp, pf) = fc.pf()
    comp_fe=pf
    fin=pf
    return fin

In [ ]:
for start, end, fin in sliding_window_analysis(rna,fold_energy):
    print(start,end,fin)

## Paso 4: Guardamos nuestros resultados en un archivo CSV

In [ ]:
def save_csv(file):
    file ='orf1ab_foldE_100.csv'
    with open(file,'w')as file_handle:
        header = "start,middle,end,folding_energy_potential\n"
        file_handle.write(header)
        for start, end, fin in sliding_window_analysis(rna,fold_energy):
            middle = (start + end ) / 2
            row = "{},{},{},{}\n".format(start, middle, end, fin)
            file_handle.write(row) 
    return(file)   
files=''
save_csv(files)

## Paso 5: Cambiamos los parámetros iniciales de simulación para  armar el modelo "Dummy"

In [ ]:
fin=0
def sliding_window_analysis(genome,function, window_size=100, step_size=1):
    """Devuelve un iterador que produce tuplas (inicio, fin, propiedad).
    Donde start y end son los índices utilizados para dividir la lista de entrada
    y function es el valor de retorno de la función dada la división
    lista.
    """
    for start in range(0, len(genome), step_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])

    for start in range(0, len(genome), window_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])
        
        
def fold_energy(genome):
    'Devuelve la energía de plegamiento de la secuencia de cada ventana'
    import RNA     
    RNA.cvar.dangles = 0
# Estructura de datos que va a pasarse a la función para provocar el emparejamiento máximo en el modelo dummy
# con dos componentes:
# 1. Un modelo 'dummy' de plegamiento para evaluar la energía de los bucles sin restricciones 
#2. Un set nuevo de parámetros de energía
    mm_data = { 'dummy': RNA.fold_compound(genome), 'params': RNA.param() }
# Nearest Neighbor Parameter reversal functions
    revert_NN = { 
        RNA.DECOMP_PAIR_HP:       lambda i, j, k, l, f, p: - f.eval_hp_loop(i, j) - 500,
        RNA.DECOMP_PAIR_IL:       lambda i, j, k, l, f, p: - f.eval_int_loop(i, j, k, l) - 500,
        RNA.DECOMP_PAIR_ML:       lambda i, j, k, l, f, p: - p.MLclosing - p.MLintern[0] - (j - i - k + l - 2) * p.MLbase - 100,
        RNA.DECOMP_ML_ML_STEM:    lambda i, j, k, l, f, p: - p.MLintern[0] - (l - k - 1) * p.MLbase,
        RNA.DECOMP_ML_STEM:       lambda i, j, k, l, f, p: - p.MLintern[0] - (j - i - k + l) * p.MLbase,
        RNA.DECOMP_ML_ML:         lambda i, j, k, l, f, p: - (j - i - k + l) * p.MLbase,
        RNA.DECOMP_ML_ML_ML:      lambda i, j, k, l, f, p: 0,
        RNA.DECOMP_ML_UP:         lambda i, j, k, l, f, p: - (j - i + 1) * p.MLbase,
        RNA.DECOMP_EXT_STEM:      lambda i, j, k, l, f, p: - f.E_ext_loop(k, l),
        RNA.DECOMP_EXT_EXT:       lambda i, j, k, l, f, p: 0,
        RNA.DECOMP_EXT_STEM_EXT:  lambda i, j, k, l, f, p: - f.E_ext_loop(i, k),
        RNA.DECOMP_EXT_EXT_STEM:  lambda i, j, k, l, f, p: - f.E_ext_loop(l, j),
        RNA.DECOMP_EXT_EXT_STEM1: lambda i, j, k, l, f, p: - f.E_ext_loop(l, j-1),
            }
# Funcion de emparejamiento máximo que llama la libreria RNA
    def MaximumMatching(i, j, k, l, d, data):
        return revert_NN[d](i, j, k, l, data['dummy'], data['params'])
# Hace un elemento compuesto 
    fc = RNA.fold_compound(genome)
# Adiciona el emparejamiento maximo entre pares de bases
    fc.sc_add_f(MaximumMatching)
    fc.sc_add_data(mm_data, None)
# llama el agoritmo propio de RNAfold para calcular la MFE
    (s, mm) = fc.mfe()
    fin=mm
    return fin


In [ ]:
for start, end, fin in sliding_window_analysis(rna,fold_energy):
    print(start,end,fin)

## Paso 6: Guardamos resultados

In [ ]:
def save_csv(file):
    file ='drunk_foldE_100.csv'
    with open(file,'w')as file_handle:
        header = "start,middle,end,folding_energy_potential\n"
        file_handle.write(header)
        for start, end, fin in sliding_window_analysis(rna,fold_energy):
            middle = (start + end ) / 2
            row = "{},{},{},{}\n".format(start, middle, end, fin)
            file_handle.write(row) 
    return(file)   
files=''
save_csv(files)

## Paso 7: Graficamos el cambio de MFE por ventana para cada resultado CSV

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('orf1ab_foldE_100.csv')
df2=pd.read_csv('drunk_foldE_100.csv')

middle_list = df.middle.tolist()
orf1ab_list = df.folding_energy_potential.tolist()
drunk_list= df2.folding_energy_potential.tolist()
x=middle_list
y=orf1ab_list
y1=drunk_list
# El eje Y es la MFE de una ventana
#El eje X es el punto medio de una ventana

plt.plot(y,'g*', y1, 'ro')


# Y ....¿Qué paso con simular?

## Paso 8: Manipulación de resultados

Ahora manipularemos nuestros resultados , para observar cambios en la estructura secundaria del ARN. Seleccionaremos solo la ventana más negativa de COVID-19 y la ventana más negativa de COVID_drunk para simular su estructura secundaria en notación de brackets, para posteriormente visualizar ambas estructuras y apreciar los cambios al mover los parámetros iniciales de la simulación

In [ ]:
# ed llama a nuestro archivo CSV de COVID-19 y nos devuelve una lista de valores
#sort los ordena de mayor a menor, sin importar que sean negativos
ed=df.sort_values('folding_energy_potential')
e=ed.head(10)
print(e)

In [ ]:
# adllama a nuestro archivo CSV de COVID dummy(drunk) y nos devuelve una lista de valores
#sort los ordena de mayor a menor, sin importar que sean negativos
ad=df2.sort_values('folding_energy_potential')
asd=ad.head(10)
print(asd)

## Paso 9: Encontramos la estructuras secundarias en notación de brackets

In [ ]:
#Computamos la estructura secundaria de RNA de COVID-19
import RNA
s=rna[318:418]

(ss, mfe) = RNA.fold(s)
# print output
print("%s\n%s (%6.2f)" % (s, ss, mfe))

In [ ]:
#Computamos la estructura secundaria de RNA del modelo dummy de COVID-19, usando los parámetros dummy
import RNA
s=rna[318:418]

seq1 = s
RNA.cvar.dangles = 0
mm_data = { 'dummy': RNA.fold_compound(seq1), 'params': RNA.param() }
revert_NN = { 
    RNA.DECOMP_PAIR_HP:       lambda i, j, k, l, f, p: - f.eval_hp_loop(i, j) - 500,
    RNA.DECOMP_PAIR_IL:       lambda i, j, k, l, f, p: - f.eval_int_loop(i, j, k, l) - 500,
    RNA.DECOMP_PAIR_ML:       lambda i, j, k, l, f, p: - p.MLclosing - p.MLintern[0] - (j - i - k + l - 2) * p.MLbase - 100,
    RNA.DECOMP_ML_ML_STEM:    lambda i, j, k, l, f, p: - p.MLintern[0] - (l - k - 1) * p.MLbase,
    RNA.DECOMP_ML_STEM:       lambda i, j, k, l, f, p: - p.MLintern[0] - (j - i - k + l) * p.MLbase,
    RNA.DECOMP_ML_ML:         lambda i, j, k, l, f, p: - (j - i - k + l) * p.MLbase,
    RNA.DECOMP_ML_ML_ML:      lambda i, j, k, l, f, p: 0,
    RNA.DECOMP_ML_UP:         lambda i, j, k, l, f, p: - (j - i + 1) * p.MLbase,
    RNA.DECOMP_EXT_STEM:      lambda i, j, k, l, f, p: - f.E_ext_loop(k, l),
    RNA.DECOMP_EXT_EXT:       lambda i, j, k, l, f, p: 0,
    RNA.DECOMP_EXT_STEM_EXT:  lambda i, j, k, l, f, p: - f.E_ext_loop(i, k),
    RNA.DECOMP_EXT_EXT_STEM:  lambda i, j, k, l, f, p: - f.E_ext_loop(l, j),
    RNA.DECOMP_EXT_EXT_STEM1: lambda i, j, k, l, f, p: - f.E_ext_loop(l, j-1),
            }
def MaximumMatching(i, j, k, l, d, data):
    return revert_NN[d](i, j, k, l, data['dummy'], data['params'])

fc = RNA.fold_compound(seq1)
fc.sc_add_f(MaximumMatching)
fc.sc_add_data(mm_data, None)
(s, mm) = fc.mfe()
print("%s\n%s (MM: %d)\n" %  (seq1, s, mm))


## Paso 10: Simulamos la estructuras secundarias 

In [ ]:
import matplotlib.pyplot as plt
import forgi.visual.mplotlib as fvm
import forgi
cg = forgi.load_rna("corona.fx", allow_many=False)
fvm.plot_rna(cg, text_kwargs={"fontweight":"black"}, lighten=0.1,
             backbone_kwargs={"linewidth":1})
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import forgi.visual.mplotlib as fvm
import forgi
cg = forgi.load_rna("drunk.fx", allow_many=False)
fvm.plot_rna(cg, text_kwargs={"fontweight":"black"}, lighten=0.1,
             backbone_kwargs={"linewidth":1})
plt.show()

## ¡Gracias!



![wohoo](Imagenes/thanks.jpg)